# Neural Network - Exercise 01

    python -m venv NN-venv
    NN-venv\Scripts\activate
    python.exe -m pip install --upgrade pip
    pip install matplotlib numpy ipykernel jupyter
    ipython kernel install --name "NN-venv" --user
    https://queirozf.com/entries/jupyter-kernels-how-to-add-change-remove

In [4]:
import numpy as np
import pandas as pd
import jupyprint.jupyprint as jp
from scipy.io import loadmat
# from graphviz import Digraph 

In [30]:
# List of flags to enable or disable parts of codes

flag_trained_network    = True
flag_my_init_pars       = True

In [31]:
# Trained Network

if flag_trained_network:
    file_name = 'Trained_Network.mat'
    mat_data = loadmat(file_name)

    weights          = mat_data['Weight']
    correct_outputs  = mat_data['correct_Output']
    epoch            = mat_data['epoch'][0][0]
    feature_inputs   = mat_data['input']

    jp('### Weights')
    print(weights)
    jp('### Correct Outputs')
    print(correct_outputs)
    jp('### Feature Inputs')
    print(feature_inputs)
    jp('### Epoch')
    print(epoch)

### Weights

[[ 9.56557803 -0.20886763 -4.57514966]]


### Correct Outputs

[[0]
 [0]
 [1]
 [1]]


### Feature Inputs

[[0 0 1]
 [0 1 1]
 [1 0 1]
 [1 1 1]]


### Epoch

10000


## Sigmoid Function

$\text{Sigmoid}(x)=\dfrac{1}{1+e^{-x}}$

$\dfrac{\partial}{\partial x}\left[\text{Sigmoid}(x)\right]=\dfrac{e^{-x}}{(1+e^{-x})^2}=\dfrac{1}{1+e^{-x}}\cdot\left(1-\dfrac{1}{1+e^{-x}}\right)$

In [32]:
# Sigmoid Function

def Sigmoid(x):
    return 1 / (1 + np.exp(-x))

## Stochastic Gradient Descent
$w'_i=w_i-\text{LR}\cdot\frac{\partial f_{cost}}{\partial w_i}=w_i-\text{LR}\cdot\frac{\partial f_{cost}}{\partial P}\cdot\frac{\partial P}{\partial t}\cdot\frac{\partial t}{\partial w_i}$ 

Con:
- $w_i$: Peso della i-esima feature
- $\text{LR}$: Learning Rate
- $f_{cost}$: Funzione di costo
- $t$: Combinazione lineare delle feature pesate 
- $P$: Combinazione lineare normalizzata con la sigmoid

In [25]:
# Stochastic Gradient Descent

def SGD_method(weight, dataset, correct_output):
# weight:           vettore dei pesi relativi alle feature
# f_input:          dataset contenente i valori delle feature per ciascun campione
# correct_output:   vettore dei valori attesi per ciascun campione

    LR = 0.9            # learning rate

    for k in range(len(weight)): 
        
        sample = dataset[k, :]                                      # ciclo sui campioni
        expected = correct_output[k]                                # prendo il valore atteso del k-esimo campione
        lin_comb = weight @ sample                                  # combinazione lineare delle feature pesate per il k-esimo campione
        predicted = Sigmoid(lin_comb)                               # normalizzazione della comb. lin. attraverso la funzione Sigmoid         
        cost_function_der = 2 * (predicted - expected)              # derivata della funzione di costo (rispetto alla variabile predicted) 
                                                                    # relativa alla metrica Squared Error Cost                         
        sigmoid_der = predicted * (1 - predicted)                   # derivata del sigmoid rispetto a lin_comb
        lc_der = sample                                             # vettore di derivate di lin_comb rispetto ai pesi w (e al bias b)

        dWeight = LR * cost_function_der * sigmoid_der * lc_der     # termine differenziale di aggiustamento per ricavare i nuovi pesi
        weight -= dWeight                                           # correzione dei pesi

    return weight

## Training

In [ ]:
# initial parameters

if flag_my_init_pars:
    weights          = None
    correct_outputs  = np.array([[0, 0, 1, 1]]).T
    epoch            = None
    feature_inputs   = np.array([[0, 0, 1],
                                 [0, 1, 1],
                                 [1, 0, 1],
                                 [1, 1, 1]])

## Testing